모듈 버전

- keras 2.2.4
- tensorflow 1.x
- anago

Google Colab - GPU 환경에서 작업

### 데이터 불러오기

- 데이터는 NE tag가 공백으로 구분되어 있다.

- *read_data* 함수를 실행하면 *file_path* 에서 텍스트 데이터를 읽고 token 리스트, tag 리스트를 반환한다.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
import pandas as pd, numpy as np

In [2]:
df = pd.read_csv('./questions/questions_corpus_bio.csv', encoding='utf-8', index_col=0)
#df = pd.read_csv('./gdrive/My Drive/Colab Notebooks/questions_corpus_bio.csv', encoding='utf-8', index_col=0)
print(df.shape)
df.head(2)

/anaconda3/envs/keras-test/lib/python3.6/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(1678295, 3)


,question,okt,okt_tag
0,계룡시 단체석 있는 복매운탕 알아요,"['계룡시', '단체', '석', '있는', '복', '매운탕', '알아요']","['B-LOC', 'B-ADJ', 'I-ADJ', 'I-ADJ', 'B-FOD', ..."
1,사천 마미밥상 무한 리필 사람들 후기 뭐에요,"['사천', '마미', '밥상', '무한', '리필', '사람', '들', '후기'...","['B-LOC', 'B-ORG', 'I-ORG', 'B-ADJ', 'I-ADJ', ..."


In [3]:
# 'okt', 'okt_tag' 컬럼을 list로 변경
df['okt'] = df['okt'].apply(lambda x: eval(x))
df['okt_tag'] = df['okt_tag'].apply(lambda x: eval(x))

In [4]:
df.head(2)

,question,okt,okt_tag
0,계룡시 단체석 있는 복매운탕 알아요,"[계룡시, 단체, 석, 있는, 복, 매운탕, 알아요]","[B-LOC, B-ADJ, I-ADJ, I-ADJ, B-FOD, I-FOD, O]"
1,사천 마미밥상 무한 리필 사람들 후기 뭐에요,"[사천, 마미, 밥상, 무한, 리필, 사람, 들, 후기, 뭐, 에요]","[B-LOC, B-ORG, I-ORG, B-ADJ, I-ADJ, O, O, O, O..."


데이터를 train과 test로 9:1의 비율로 나눈다.
 - *train* -> 모델 훈련;
 - *test* -> 모델 평가와 hyperparameter 조정

In [0]:
from sklearn.model_selection import train_test_split

train_tokens, test_tokens, train_tags, test_tags = \
    train_test_split(df['okt'].values
                    ,df['okt_tag'].values
                    ,test_size=0.1
                    ,random_state=42)

In [0]:
train_tokens.shape, train_tags.shape, test_tokens.shape, test_tags.shape

((1510465,), (1510465,), (167830,), (167830,))

In [0]:
train_tokens[:3]

array([list(['치맥', '타운', '사람', '들', '반응', '알려줘요']),
       list(['페리', '아', '상세', '주소', '알아보려고요']),
       list(['키친', '홀릭', '핵', '맛있는', '평', '알', '고', '계신가요'])],
      dtype=object)

In [0]:
train_tags[:3]

array([list(['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O']),
       list(['B-ORG', 'I-ORG', 'O', 'O', 'O']),
       list(['B-ORG', 'I-ORG', 'B-ADJ', 'I-ADJ', 'O', 'O', 'O', 'O'])],
      dtype=object)

In [0]:
for i in range(0,2):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))

치맥	B-ORG
타운	I-ORG
사람	O
들	O
반응	O
알려줘요	O
페리	B-ORG
아	I-ORG
상세	O
주소	O
알아보려고요	O


In [5]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [0]:
train_tokens,train_tags

(array([list(['치맥', '타운', '사람', '들', '반응', '알려줘요']),
        list(['페리', '아', '상세', '주소', '알아보려고요']),
        list(['키친', '홀릭', '핵', '맛있는', '평', '알', '고', '계신가요']), ...,
        list(['당진', '순이', '네', '밥집', '솔직', '후기', '궁금합니다']),
        list(['폭포', '있는', '환상', '적', '인', '추천', '할', '데', '어디', '있죠']),
        list(['청도군', '서비스', '는', '덤', '인', '스페인', '좋은', '데', '어디', '인데', '요'])],
       dtype=object), array([list(['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O']),
        list(['B-ORG', 'I-ORG', 'O', 'O', 'O']),
        list(['B-ORG', 'I-ORG', 'B-ADJ', 'I-ADJ', 'O', 'O', 'O', 'O']),
        ..., list(['B-LOC', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O']),
        list(['B-ADJ', 'I-ADJ', 'B-ADJ', 'I-ADJ', 'I-ADJ', 'O', 'O', 'O', 'O', 'O']),
        list(['B-LOC', 'B-ADJ', 'I-ADJ', 'I-ADJ', 'I-ADJ', 'B-FOD', 'O', 'O', 'O', 'O', 'O'])],
       dtype=object))

In [6]:
from anago.tagger import Tagger
from anago.trainer import Trainer
from anago.wrapper import Sequence
from anago.utils import NERSequence

/anaconda3/envs/keras-test/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
# 약 1시간 소요
model.fit(x_train=train_tokens
         ,y_train=train_tags)

Epoch 1/1





47203/47203 [==============================] - 4254s 90ms/step - loss: 1.7863


In [0]:
model.score(test_tokens, test_tags)

0.9986188238857988

In [0]:
text = '파스쿠찌 위치 알려주세요'
model.analyze(text)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '파스쿠찌',
   'type': 'ORG'}],
 'words': ['파스쿠찌', '위치', '알려주세요']}

In [0]:
text = '담소 순대국밥집 청결한가요?'
model.analyze(text)

{'entities': [{'beginOffset': 0,
   'endOffset': 2,
   'score': 1.0,
   'text': '담소 순대국밥집',
   'type': 'ORG'}],
 'words': ['담소', '순대국밥집', '청결한가요?']}

In [0]:
text = '담소 순대국밥집 맛있어요?'
model.analyze(text)

{'entities': [{'beginOffset': 0,
   'endOffset': 2,
   'score': 1.0,
   'text': '담소 순대국밥집',
   'type': 'ORG'}],
 'words': ['담소', '순대국밥집', '맛있어요?']}

In [0]:
text = '담소 순대국밥집 맛있는 곳인가요?'
model.analyze(text)

{'entities': [{'beginOffset': 0,
   'endOffset': 2,
   'score': 1.0,
   'text': '담소 순대국밥집',
   'type': 'ORG'}],
 'words': ['담소', '순대국밥집', '맛있는', '곳인가요?']}

In [0]:
text = '숯불구이 맛있는곳 알려주실래요?'
model.analyze(text)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '숯불구이',
   'type': 'ORG'}],
 'words': ['숯불구이', '맛있는곳', '알려주실래요?']}

In [0]:
from konlpy.tag import Okt
okt = Okt()
okt.morphs(text)

['숯불', '구이', '맛있는', '곳', '알려주실래요', '?']

In [0]:
# 원래는 model.analyze(text, tokenizer=okt.morphs)로 쓰면 되는데 이 코드가 안먹혀서 아래와 같이 사용

text = '숯불구이 맛있는곳 알려주실래요?'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 2,
   'score': 1.0,
   'text': '숯불 구이',
   'type': 'FOD'}],
 'words': ['숯불', '구이', '맛있는', '곳', '알려주실래요', '?']}

In [0]:
text = '온센텐동 사람들 리뷰 좀 알려주세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 2,
   'score': 1.0,
   'text': '온센 텐동',
   'type': 'ORG'}],
 'words': ['온센', '텐동', '사람', '들', '리뷰', '좀', '알려주세요']}

In [0]:
text = '온센텐동 위치가 어떻게 되나요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 2,
   'score': 1.0,
   'text': '온센 텐동',
   'type': 'ORG'}],
 'words': ['온센', '텐동', '위치', '가', '어떻게', '되나요']}

In [0]:
text = '파스쿠찌 맛있어요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '파스쿠찌',
   'type': 'ORG'}],
 'words': ['파스쿠찌', '맛있어요']}

In [0]:
text = '가성비 대박인 육개장 집 알려주세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 4,
   'score': 1.0,
   'text': '가성 비 대박 인',
   'type': 'ADJ'},
  {'beginOffset': 4,
   'endOffset': 6,
   'score': 1.0,
   'text': '육개장 집',
   'type': 'FOD'}],
 'words': ['가성', '비', '대박', '인', '육개장', '집', '알려주세요']}

In [0]:
text = '맛있는 육개장집 알려주세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '맛있는',
   'type': 'ADJ'},
  {'beginOffset': 1,
   'endOffset': 3,
   'score': 1.0,
   'text': '육개장 집',
   'type': 'FOD'}],
 'words': ['맛있는', '육개장', '집', '알려주세요']}

In [0]:
text = '육개장 맛있는 곳 알려주세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '육개장',
   'type': 'FOD'}],
 'words': ['육개장', '맛있는', '곳', '알려주세요']}

In [0]:
text = '서울에 맛있는 육개장 가게 알려주세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '서울',
   'type': 'LOC'},
  {'beginOffset': 1,
   'endOffset': 3,
   'score': 1.0,
   'text': '에 맛있는',
   'type': 'ADJ'},
  {'beginOffset': 3,
   'endOffset': 5,
   'score': 1.0,
   'text': '육개장 가게',
   'type': 'FOD'}],
 'words': ['서울', '에', '맛있는', '육개장', '가게', '알려주세요']}

In [0]:
text = '제주도에 유명한 흑돼지 식당 알려주세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '제주도',
   'type': 'LOC'},
  {'beginOffset': 1,
   'endOffset': 6,
   'score': 1.0,
   'text': '에 유명한 흑 돼지 식당',
   'type': 'ORG'}],
 'words': ['제주도', '에', '유명한', '흑', '돼지', '식당', '알려주세요']}

In [0]:
text = '제주도에 맛있는 흑돼지 식당 알려주세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '제주도',
   'type': 'LOC'},
  {'beginOffset': 1,
   'endOffset': 6,
   'score': 1.0,
   'text': '에 맛있는 흑 돼지 식당',
   'type': 'ORG'}],
 'words': ['제주도', '에', '맛있는', '흑', '돼지', '식당', '알려주세요']}

In [0]:
text = '제주도에 흑돼지 식당 맛있는 곳 알려주세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '제주도',
   'type': 'LOC'},
  {'beginOffset': 1,
   'endOffset': 5,
   'score': 1.0,
   'text': '에 흑 돼지 식당',
   'type': 'FOD'}],
 'words': ['제주도', '에', '흑', '돼지', '식당', '맛있는', '곳', '알려주세요']}

In [0]:
text = '제주도에 흑돼지 식당 맛집 알려주세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '제주도',
   'type': 'LOC'},
  {'beginOffset': 1,
   'endOffset': 5,
   'score': 1.0,
   'text': '에 흑 돼지 식당',
   'type': 'FOD'}],
 'words': ['제주도', '에', '흑', '돼지', '식당', '맛집', '알려주세요']}

In [0]:
text = '제주도에 흑돼지 식당 맛 개쩌는 곳 알려주세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '제주도',
   'type': 'LOC'},
  {'beginOffset': 1,
   'endOffset': 6,
   'score': 1.0,
   'text': '에 흑 돼지 식당 맛',
   'type': 'FOD'}],
 'words': ['제주도', '에', '흑', '돼지', '식당', '맛', '개쩌', '는', '곳', '알려주세요']}

In [0]:
# Okt는 맛있고를 '맛있고'로 형태소 분석, Mecab은 맛있고를 '맛있''고'로 형태소 분석한다.
# train 데이터에 '맛있는'은 있지만 '맛있고'는 없다. 
# 그래서 맛있고 깔끔한을 2개의 형용사가 아닌 하나의 형용사로 분류해버리는 오류가 난 것 같다.
# 아래 문장 테스트를 통해 NER 모델 생성 시 Okt가 아닌 Mecab을 써야 할 필요성을 느꼈다.

text = '맛있고 깔끔한 한식집 아세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 2,
   'score': 1.0,
   'text': '맛있고 깔끔한',
   'type': 'ADJ'},
  {'beginOffset': 2,
   'endOffset': 4,
   'score': 1.0,
   'text': '한식 집',
   'type': 'FOD'}],
 'words': ['맛있고', '깔끔한', '한식', '집', '아세요']}

In [0]:
text = '맛있는 깔끔한 한식집 아세요'
text_prepared = ' '.join(word for word in okt.morphs(text))
model.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '맛있는',
   'type': 'ADJ'},
  {'beginOffset': 1,
   'endOffset': 2,
   'score': 1.0,
   'text': '깔끔한',
   'type': 'ADJ'},
  {'beginOffset': 2,
   'endOffset': 4,
   'score': 1.0,
   'text': '한식 집',
   'type': 'FOD'}],
 'words': ['맛있는', '깔끔한', '한식', '집', '아세요']}

In [0]:
# 모델 저장 (weights,params,preprocessor로 나눠서 저장)
model.save('./gdrive/My Drive/Colab Notebooks/model_weights'
          ,'./gdrive/My Drive/Colab Notebooks/model_params'
          ,'./gdrive/My Drive/Colab Notebooks/model_preprocessor')

In [8]:
import tensorflow as tf
tf.version

<module 'tensorflow_core._api.v2.version' from '/anaconda3/envs/keras-test/lib/python3.6/site-packages/tensorflow_core/_api/v2/version/__init__.py'>

In [7]:
# 모델 불러오기 테스트
weights = './model_weights'
params = './model_params'
preprocessor = './model_preprocessor'
tm = Sequence.load(weights, params, preprocessor)

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [0]:
tm.analyze(text_prepared)

{'entities': [{'beginOffset': 0,
   'endOffset': 1,
   'score': 1.0,
   'text': '맛있는',
   'type': 'ADJ'},
  {'beginOffset': 1,
   'endOffset': 2,
   'score': 1.0,
   'text': '깔끔한',
   'type': 'ADJ'},
  {'beginOffset': 2,
   'endOffset': 4,
   'score': 1.0,
   'text': '한식 집',
   'type': 'FOD'}],
 'words': ['맛있는', '깔끔한', '한식', '집', '아세요']}